I restructure a bit of the code here and also some of the data. The researchers have not been consistent in the enumeration of the peptides neither the domains so its kinda difficult to structure it all together. 

**UPDATE**:

There was lots of redundant code that I had written for demonstration purposes. The code written henceforth is cleaner and more modular(I hope) 

In [1]:
import os 
os.chdir('E:\Ecole\Year 3\Projet 3A')
import pandas as pd
import numpy as np 

class Domain:
    
    def __init__(self, name):
        self.name = name
        self.thresholds = None
        self.thetas = None

class Peptide:
    
    def __init__(self, name):
        self.name = name
        self.sequence = None
        self.sequence_bis = None ##Sequence bis are the last five amino acids
        self.energy_ground = 0.0 ##Anticipating the calculation of a ground state energy for the peptide
        
class Data:
    
    def __init__(self):
        temp_df = pd.read_excel('Data_PDZ/MDSM_01_stiffler_bis.xls')
        self.aminoacids = [acid.encode('utf-8') for acid in list(temp_df.columns[:20])]
        self.df = temp_df.T
        self.domains = [Domain(domain.encode('utf-8')) for domain in list(self.df.columns)]
        self.domain_names = [domain.name for domain in self.domains]
        self.pep_seqs = []
        self.pep_names = []
        with open('Data_PDZ/peptides.free') as f:
            for line in f:
                x = line.split()
                self.pep_seqs.append(x[1])
                self.pep_names.append(x[0])
        self.peptides = [Peptide(name) for name in self.pep_names]
        
    def create_domains(self):
        for domain in self.domains:
            domain.thetas = self.df[domain.name][:100]
            domain.thetas = np.asarray(domain.thetas)
            domain.thetas = domain.thetas.reshape(5,20)
            domain.thresholds = np.asarray(self.df[domain.name][100:])   
    
    def create_peptides(self):
        for i in range(len(self.pep_seqs)):
            self.peptides[i].sequence = self.pep_seqs[i]
            self.peptides[i].sequence_bis = list(self.pep_seqs[i])[5:]        

In [2]:
PDZ_Data = Data()
PDZ_Data.create_domains()
PDZ_Data.create_peptides()

Now we have created the preliminary data with the binding energy values and the peptide sequences. The last thing left to do is to get the data from the interaction matrix for each of the domain. 

We also write some convenience functions like the log and sigmoid functions 

In [3]:
fp_interaction_matrix = pd.read_excel('Data_PDZ/fp_interaction_matrix.xlsx')
for column in fp_interaction_matrix.columns:
    fp_interaction_matrix.loc[fp_interaction_matrix[column] == 0.0, column] = -1.0
fp_interaction_matrix = fp_interaction_matrix.rename(columns=lambda x: str(x).replace(" ", ""))

In [4]:
## Sigmoid Function
def sigmoid(x, a=1):
    return 1.0/(1+np.exp(-1.0*a*x))
## Log(1+exp(-x)) 
## We take care of numerical stability for values of x < 0
def log_modified(x):
    if x > 0:
        return np.log(1+np.exp(-x))
    else:
        return -x + np.log(1+np.exp(x))
    
## Convenience functions to convert between letter sequences and indexed sequences. 
## The index for each amino acid is computed using the enumeration presented in the file "MDSM_01_stiffler_bis.xls" 
def convert2seq(seq_int):
    return [PDZ_Data.aminoacids[i] for i in seq_int]
def convert2int(seq_pep):
    return [PDZ_Data.aminoacids.index(pep) for pep in seq_pep]

In [5]:
## Unlike the previous code, where we had two different versions of evaluate_score function, here we have just one. 
## To enter the sequence of a known peptide use convert2int(peptide name) 
def eval_score(domain, sequence):
    score = 0.0
    for i in range(5):
        score += domain.thetas[i,sequence[i]]
    return score - domain.thresholds[0]

In [6]:
## Similarly for the energy evaluation we simply use the single version which use the sequence directly 
## The sequence argument will be useful when making mutations starting from the basic peptide sequence 
def eval_energy(peptide, sequence, verbose=0):
    score_natural = 0.0
    energies = []
    for i in range(len(PDZ_Data.domain_names)): 
        temp = eval_score(PDZ_Data.domains[i], sequence)
        alpha = fp_interaction_matrix[peptide.name][i]
        if alpha > 0:
            alpha = +1.0
        score = temp*alpha
        temp2 = log_modified(score)
        energies.append({'Energy': temp2, 'alpha': alpha, 'score': temp})
        score_natural += temp2 
    if verbose == 0:
        return score_natural
    else:
        return score_natural, energies

Let us take a particular example and illustrate the use of these basic functions on them. We shall use the protein *Caspr2*

In [7]:
ix = PDZ_Data.pep_names.index('Caspr2')
print ix
pep_demo = PDZ_Data.peptides[ix]
print pep_demo.name
print pep_demo.sequence
print pep_demo.sequence_bis

4
Caspr2
IDESKKEWLI
['K', 'E', 'W', 'L', 'I']


In [8]:
print eval_energy(pep_demo, convert2int(pep_demo.sequence_bis))
score, energies = eval_energy(pep_demo, convert2int(pep_demo.sequence_bis), verbose=1)

10.2327578819


In [9]:
neg = []
pos = [] 
for i in range(len(energies)):
    if energies[i]['alpha'] == -1.0:
        neg.append({'data':energies[i], 'name': PDZ_Data.domain_names[i], 'index' : i+2})
    else:
        pos.append({'data':energies[i], 'name': PDZ_Data.domain_names[i], 'index' : i+2})
for item in pos:
    print item

{'index': 2, 'data': {'alpha': 1.0, 'Energy': 0.58841515662593658, 'score': 0.22172999999999998}, 'name': 'Cipp (03/10)'}
{'index': 8, 'data': {'alpha': 1.0, 'Energy': 0.69316718075994543, 'score': -4.0000000000262048e-05}, 'name': 'Dlgh3 (1/1)'}
{'index': 19, 'data': {'alpha': 1.0, 'Energy': 2.5693183193875501e-05, 'score': 10.569272000000002}, 'name': 'HtrA3 (1/1)'}
{'index': 33, 'data': {'alpha': 1.0, 'Energy': 0.6931021815724453, 'score': 9.0000000000145519e-05}, 'name': 'Mpp7 (1/1)'}
{'index': 43, 'data': {'alpha': 1.0, 'Energy': 0.69316718075994543, 'score': -4.0000000000262048e-05}, 'name': 'PAR-3 (3/3)'}


In [10]:
## We calculate the natural energies for each of the peptides 
for pep in PDZ_Data.peptides:
    pep.energy_ground = eval_energy(pep, convert2int(pep.sequence_bis))

In [11]:
## Sanity Check 
print PDZ_Data.peptides[ix].energy_ground

10.2327578819



## Simulation 
Now we shall start with the real Monte Carlo step. Our algorithm is based on the famous Metropolis algorithm. We start with a given peptide and its sequence. To each peptide is associated a particular energy(which we calculated above). We expect that under mutations of the sequence, this energy will change. Depending on whether the energy changes or not after a point mutation, we shall accept or reject the mutation. 

Let us first start off by writing some convenience functions to make point mutations

**UPDATE**:

After some problems with the data, mainly to do with the way the data was indexed by the researchers, we are finally in a position to retrieve something meaningful out of the data. 

For an initial run, the acceptance rule for the mutations was rather simple: If the energy reduced we would accept the mutation, otherwise not. Surprisingly, with such a rule, no new mutations which conserve the original binding pairs, are allowed. Further exploration is needed at this stage to completely deconstruct this finding.

To facilitate further analysis, I compute the energies and the scores for each of the peptide-PDZ pairs. These energies will serve as a reference for further mutations. 

To make a mutation we need two numbers, one a number between 0 and 4 which will tell us the position to be mutated and a number between 0 and 19 which will tell us the amino acid to put in that position. We can do this easily by making two calls to the randomint function in numpy. 

In [12]:
y = np.random.randint(5)
z = np.random.randint(20)
print y, z

0 2


This is a basic run where we introduce mutations and see whether the energy actually reduces or not. The base energy is 8.178291

We now run the Metropolis Hastings algorithm for our data set. If the energy decreases then it is a favorable mutation and we accept the mutation, otherwise we reject the mutation

In [15]:
test_peptide = PDZ_Data.peptides[57]
print test_peptide.name
print test_peptide.energy_ground

AcvR1
8.1782910361


In [16]:
print " Uniform {} Test {}".format(np.random.uniform(), 2)

 Uniform 0.38313776258 Test 2


In [17]:
def run_mc(nb_runs, peptide, temp=0, nb_cycles=10):
    sims = []
    base_seq = convert2int(peptide.sequence_bis)
    for j in range (nb_cycles):
        print "\n Cycle number : {}\n".format(j+1)
        sim_results = []
        mutated_sequences = []
        mutated_energies = []
        mut_seq = base_seq
        mut_energy = test_peptide.energy_ground
        for i in range(nb_runs):
            y = np.random.randint(5)
            z = np.random.randint(20)
            temp_seq = mut_seq
            temp_seq[y] = z
            temp_energy = eval_energy(test_peptide, temp_seq)
            if temp == 0:
                if temp_energy < mut_energy:
                    mut_energy = temp_energy
                    mut_seq = temp_seq
                    sim_results.append({'Sequence': temp_seq, 'Energy': temp_energy, 'Accepted': 1})
                    print "Accepted {} {} {} ".format(temp_seq, temp_energy, convert2seq(temp_seq))
                else:
                    sim_results.append({'Sequence': temp_seq, 'Energy': temp_energy, 'Accepted': 0})  
            else:
                ratio = np.exp(-temp*(temp_energy-mut_energy))
                prob_trans = min(1, ratio)
                x = np.random.uniform()
                
                if x < prob_trans:
                    mut_energy = temp_energy
                    mut_seq = temp_seq
                    print "Run number: {}\n".format(i)
                    print "Uniform {} Ratio {} Prob_Trans {} ".format(x,ratio,prob_trans)
                    sim_results.append({'Sequence': temp_seq, 'Energy': temp_energy, 'Accepted': 1})
                    print "Accepted {} {} {} \n".format(temp_seq, temp_energy, convert2seq(temp_seq))
                else:
                    sim_results.append({'Sequence': temp_seq, 'Energy': temp_energy, 'Accepted': 0})  
                
                ##print "Rejected {} {}".format(temp_seq, temp_energy)
            mutated_sequences.append(temp_seq)
            mutated_energies.append(temp_energy)
        sims.append({'Results' : sim_results, 'Mutated sequences': mutated_sequences, 'Mutated Energies': mutated_energies}) 
    return sims

Here we perform mutations for the given peptide and only accept those mutations which lead to a reduction in the energy. Here the inverse temperature $\beta$ is infinite (or $T=0$)

In [18]:
results = run_mc(100, test_peptide)


 Cycle number : 1

Accepted [3, 15, 13, 18, 14] 8.13511212421 ['L', 'K', 'Y', 'D', 'C'] 
Accepted [18, 15, 3, 9, 14] 4.9491783242 ['D', 'K', 'L', 'S', 'C'] 
Accepted [1, 9, 14, 14, 15] 3.04440580312 ['A', 'S', 'C', 'C', 'K'] 
Accepted [15, 9, 0, 14, 15] 2.26956269709 ['K', 'S', 'G', 'C', 'K'] 
Accepted [6, 19, 12, 19, 15] 1.49104278243 ['P', 'E', 'Q', 'E', 'K'] 

 Cycle number : 2

Accepted [3, 9, 12, 0, 14] 4.39105096441 ['L', 'S', 'Q', 'G', 'C'] 
Accepted [8, 10, 16, 3, 0] 4.37372507888 ['W', 'T', 'R', 'L', 'G'] 
Accepted [14, 6, 3, 11, 2] 4.01280292609 ['C', 'P', 'L', 'N', 'V'] 
Accepted [14, 6, 3, 11, 8] 3.15284637563 ['C', 'P', 'L', 'N', 'W'] 
Accepted [15, 6, 3, 11, 8] 1.80156682588 ['K', 'P', 'L', 'N', 'W'] 
Accepted [15, 10, 16, 10, 12] 1.32794572805 ['K', 'T', 'R', 'T', 'Q'] 

 Cycle number : 3

Accepted [2, 14, 2, 1, 4] 5.47853996319 ['V', 'C', 'V', 'A', 'I'] 
Accepted [11, 11, 2, 1, 4] 4.53352776239 ['N', 'N', 'V', 'A', 'I'] 
Accepted [11, 11, 2, 1, 0] 2.61365328132 ['N', '

We shall now run the simulation with a rejection probability proportional to $e^{-\beta(E_{new} - E_{old})}$ where $\beta$ is the inverse temperature and $E_{new}$ and $E_{old}$ are the energies after and before the mutation. 

In [19]:
results_2 = run_mc(100, test_peptide, temp=1.0, nb_cycles=1)


 Cycle number : 1

Run number: 6

Uniform 0.0941514533946 Ratio 45.8820898444 Prob_Trans 1 
Accepted [3, 1, 18, 18, 0] 4.35221619465 ['L', 'A', 'D', 'D', 'G'] 

Run number: 7

Uniform 0.469734413894 Ratio 5.97604458196 Prob_Trans 1 
Accepted [3, 1, 18, 18, 6] 2.56445728668 ['L', 'A', 'D', 'D', 'P'] 

Run number: 9

Uniform 0.61344689297 Ratio 0.853102683495 Prob_Trans 0.853102683495 
Accepted [11, 1, 18, 18, 15] 2.72333264617 ['N', 'A', 'D', 'D', 'K'] 

Run number: 10

Uniform 0.671843768668 Ratio 1.0188726286 Prob_Trans 1 
Accepted [11, 1, 9, 18, 15] 2.70463589621 ['N', 'A', 'S', 'D', 'K'] 

Run number: 11

Uniform 0.202986571506 Ratio 1.0 Prob_Trans 1 
Accepted [11, 1, 9, 18, 15] 2.70463589621 ['N', 'A', 'S', 'D', 'K'] 

Run number: 12

Uniform 0.021194549214 Ratio 1.84736074468 Prob_Trans 1 
Accepted [0, 1, 9, 18, 15] 2.09087790021 ['G', 'A', 'S', 'D', 'K'] 

Run number: 64

Uniform 0.0191867125575 Ratio 0.0742720810223 Prob_Trans 0.0742720810223 
Accepted [15, 12, 8, 9, 9] 4.69089

Let us now consider a peptide which according to the FP Interaction Matrix binds to a lot many domains and see how its behavior is different from that of the peptide *AcvR1*. For the next demo, we shall consider *Cnksr2*

In [20]:
test_peptide2 = PDZ_Data.peptides[PDZ_Data.pep_names.index('Cnksr2')]
print test_peptide2.name
print test_peptide2.energy_ground

Cnksr2
6.91618770844


In [21]:
results_3 = run_mc(100, test_peptide2)


 Cycle number : 1

Accepted [4, 16, 3, 4, 8] 5.41653824096 ['I', 'R', 'L', 'I', 'W'] 
Accepted [6, 18, 1, 12, 10] 3.75345252862 ['P', 'D', 'A', 'Q', 'T'] 
Accepted [6, 18, 1, 12, 14] 3.46545366948 ['P', 'D', 'A', 'Q', 'C'] 
Accepted [12, 14, 11, 2, 8] 3.16711544281 ['Q', 'C', 'N', 'V', 'W'] 
Accepted [6, 14, 6, 5, 8] 1.79672660699 ['P', 'C', 'P', 'M', 'W'] 

 Cycle number : 2

Accepted [14, 1, 9, 1, 18] 2.15421747697 ['C', 'A', 'S', 'A', 'D'] 
Accepted [16, 1, 9, 18, 1] 1.98779237214 ['R', 'A', 'S', 'D', 'A'] 

 Cycle number : 3

Accepted [11, 17, 17, 3, 12] 5.20939731745 ['N', 'H', 'H', 'L', 'Q'] 
Accepted [11, 17, 12, 3, 12] 4.0900049541 ['N', 'H', 'Q', 'L', 'Q'] 
Accepted [11, 17, 12, 1, 12] 2.4095296941 ['N', 'H', 'Q', 'A', 'Q'] 
Accepted [15, 19, 16, 15, 6] 2.24172962032 ['K', 'E', 'R', 'K', 'P'] 
Accepted [14, 19, 3, 6, 19] 1.8063688857 ['C', 'E', 'L', 'P', 'E'] 

 Cycle number : 4

Accepted [18, 14, 15, 0, 4] 4.58317294247 ['D', 'C', 'K', 'G', 'I'] 
Accepted [2, 9, 6, 4, 5] 4.1

In [22]:
results_4 = run_mc(100, test_peptide2, temp=1.0, nb_cycles=1)


 Cycle number : 1

Run number: 10

Uniform 0.468596423965 Ratio 0.580379214895 Prob_Trans 0.580379214895 
Accepted [2, 4, 16, 15, 11] 8.7223646064 ['V', 'I', 'R', 'K', 'N'] 

Run number: 11

Uniform 0.604241988319 Ratio 6.01827493495 Prob_Trans 1 
Accepted [2, 9, 16, 15, 11] 6.92756394381 ['V', 'S', 'R', 'K', 'N'] 

Run number: 13

Uniform 0.300685499396 Ratio 3.98169676272 Prob_Trans 1 
Accepted [1, 9, 16, 15, 11] 5.54585589307 ['A', 'S', 'R', 'K', 'N'] 

Run number: 14

Uniform 0.0178981621224 Ratio 1.15160642327 Prob_Trans 1 
Accepted [1, 15, 16, 15, 11] 5.40469803564 ['A', 'K', 'R', 'K', 'N'] 

Run number: 30

Uniform 0.0597519686676 Ratio 1.86567626945 Prob_Trans 1 
Accepted [0, 1, 17, 6, 8] 4.7810744373 ['G', 'A', 'H', 'P', 'W'] 

Run number: 31

Uniform 0.0931672744717 Ratio 1.23126419537 Prob_Trans 1 
Accepted [0, 5, 17, 6, 8] 4.57303299463 ['G', 'M', 'H', 'P', 'W'] 

Run number: 32

Uniform 0.056232560619 Ratio 5.9157016632 Prob_Trans 1 
Accepted [0, 17, 17, 6, 8] 2.795422879